Copyright **`(c)`** 2024 Giovanni Squillero `<squillero@polito.it>`  
`https://github.com/squillero/computational-intelligence`  
Free for personal or classroom use; see 'LICENCE.md' for details.

In [53]:
import random
import pandas as pd
import math
from tqdm import tqdm 
import numpy as np
import geopy.distance


In [54]:
#change cities here
cities=pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
cities = {row['name']: (row['lat'], row['lon']) for index, row in cities.iterrows()}


In [55]:
nomi_città = list(cities.keys())

# Precompute the distance matrix
dist_matrix = np.zeros((len(nomi_città), len(nomi_città)))

for i, c1 in enumerate(nomi_città):
    for j, c2 in enumerate(nomi_città):
        if i != j:
            dist_matrix[i, j] = geopy.distance.geodesic(cities[c1], cities[c2]).km

# Function to get the distance from the precomputed matrix
def distanza(c1: str, c2: str) -> float:
    i, j = nomi_città.index(c1), nomi_città.index(c2)
    return dist_matrix[i, j]

# Calculate the cost of a route
def costo(percorso: list[str]) -> float:
    costo_totale = sum(distanza(percorso[i], percorso[i + 1]) for i in range(len(percorso) - 1))
    costo_totale += distanza(percorso[0], percorso[-1])  # Close the loop
    return costo_totale

def is_valid_solution(percorso: list[str]) -> bool:

    # Compare the length of the route with the number of cities
    if len(percorso) != len(nomi_città):
        return False

    # Check if all cities are present in the route
    return set(percorso) == set(nomi_città)


## Greedy 1°

In [56]:

def percorso_greedy(partenza: str) -> list[str]:
    percorso = [partenza]
    città_da_visitare = set(cities.keys())
    città_da_visitare.remove(partenza)

    while città_da_visitare:
        ultima_città = percorso[-1]
        prossima_città = min(città_da_visitare, key=lambda c: distanza(ultima_città, c))
        percorso.append(prossima_città)
        città_da_visitare.remove(prossima_città)
    return percorso




città_di_partenza = list(cities.keys())[0]


# find path  greedy
miglior_percorso = percorso_greedy(città_di_partenza)

print("Best path (greedy):", miglior_percorso)
print("Path cost:", costo(miglior_percorso))





Best path (greedy): ['Isangel', 'Vila', 'Lakatoro', 'Norsup', 'Luganville', 'Port Olry', 'Longana', 'Sola']
Path cost: 1475.528091104531


#### greedy + 2opt

In [57]:

def ottimizza_2opt(percorso: list[str]) -> list[str]:
    migliorato = True
    while migliorato:
        migliorato = False
        for i in range(1, len(percorso) - 1):
            for j in range(i + 1, len(percorso)):
                if j - i == 1:  # Avoid consecutive swaps
                    continue
                nuovo_percorso = percorso[:i] + percorso[i:j][::-1] + percorso[j:]
                if costo(nuovo_percorso) < costo(percorso):
                    percorso = nuovo_percorso
                    migliorato = True
    return percorso


def ottimizza_3opt(percorso: list[str]) -> list[str]:
    migliorato = True
    while migliorato:
        migliorato = False
        n = len(percorso)
        for i in range(n - 2):
            for j in range(i + 2, n - 1):
                for k in range(j + 2, n if i > 0 else n - 1):
                     # Generate the three possible new paths
                    nuovo_percorso1 = percorso[:i] + percorso[i:j][::-1] + percorso[j:k][::-1] + percorso[k:]
                    nuovo_percorso2 = percorso[:i] + percorso[j:k] + percorso[i:j] + percorso[k:]
                    nuovo_percorso3 = percorso[:i] + percorso[j:k][::-1] + percorso[i:j][::-1] + percorso[k:]
                    
                    # Check if one of the new paths is better
                    for nuovo_percorso in [nuovo_percorso1, nuovo_percorso2, nuovo_percorso3]:
                        if costo(nuovo_percorso) < costo(percorso):
                            percorso = nuovo_percorso
                            migliorato = True
    return percorso


miglior_percorso = ottimizza_2opt(miglior_percorso)

costo_totale = costo(miglior_percorso)


print("Best path (greedy + opt):", miglior_percorso)
print("Path cost:", costo_totale)


Best path (greedy + opt): ['Isangel', 'Vila', 'Lakatoro', 'Norsup', 'Luganville', 'Port Olry', 'Longana', 'Sola']
Path cost: 1475.528091104531


## Lin-Kernighan

In [58]:
def lin_kernighan(cities, max_iterations=10000):
    current_solution = list(cities.keys())
    current_cost = costo(current_solution)

    improved = True
    while improved and max_iterations > 0:
        improved = False
        
        for i in range(len(current_solution)):
            for j in range(i + 2, len(current_solution)):
                # Calculate the cost of the new path after the swap
                new_cost = (current_cost -
                             distanza(current_solution[i], current_solution[i + 1]) -
                             distanza(current_solution[j], current_solution[(j + 1) % len(current_solution)]) +
                             distanza(current_solution[i], current_solution[j]) +
                             distanza(current_solution[i + 1], current_solution[(j + 1) % len(current_solution)]))

                # Check if the new cost is better
                if new_cost < current_cost:
                    # Perform the swap in-place
                    current_solution[i + 1:j + 1] = reversed(current_solution[i + 1:j + 1])
                    current_cost = new_cost
                    improved = True
                    break  # Exit the inner loop if an improvement was found
            
            if improved:
                break  # Exit the outer loop if an improvement was found

        max_iterations -= 1  # Decrement the iteration counter

    return current_solution, current_cost

# Run the algorithm
final_path, final_cost = lin_kernighan(cities)
print(is_valid_solution(final_path))
print("Best path (Lin-Kernighan):", final_path)
print("Path cost:", costo(final_path))


True
Best path (Lin-Kernighan): ['Isangel', 'Longana', 'Sola', 'Port Olry', 'Luganville', 'Norsup', 'Lakatoro', 'Vila']
Path cost: 1345.5449564733115


## genetic

In [59]:
# Parameters of the genetic algorithm
POPULATION_SIZE = 150
NUM_GENERATIONS = 500
INITIAL_MUTATION_RATE = 0.5  # Initial mutation rate
MIN_MUTATION_RATE = 0.1      # Minimum mutation rate
SELECTION_SIZE = 30
ELITISM_SIZE = 5             # Number of elite individuals to keep (5/10% of the population)

# Function to create a random path
def crea_percorso_casuale(città: list[str]) -> list[str]:
    percorso = città[:]
    random.shuffle(percorso)
    return percorso

# Function to create an initial population
def crea_popolazione(città: list[str], size: int) -> list[list[str]]:
    
    popolazione=[]
    for _ in range(size):
        popolazione.append(crea_percorso_casuale(città))
    return popolazione

# Function to select an individual using tournament selection
def selezione_torneo(popolazione: list[list[str]], size: int) -> list[str]:
    partecipanti = random.sample(popolazione, size)
    return min(partecipanti, key=lambda percorso: costo(percorso))

# Function to select an individual using rank-based selection
def selezione_ranking(popolazione: list[list[str]]) -> list[str]:
    popolazione_ordinata = sorted(popolazione, key=lambda percorso: costo(percorso))
    rank_weights = [1 / (i + 1) for i in range(len(popolazione_ordinata))]
    somma_pesi = sum(rank_weights)
    probabilità = [peso / somma_pesi for peso in rank_weights]
    return random.choices(popolazione_ordinata, weights=probabilità, k=1)[0]

# Hybrid PMX-OX crossover function
def pmx_ox_crossover(parent1: list[str], parent2: list[str]) -> list[str]:
    size = len(parent1)
    child = [-1] * size
    start, end = sorted(random.sample(range(size), 2))
    
    # PMX: Copy segment from the first parent
    child[start:end] = parent1[start:end]
    
    # OX: Map values from the second parent to the copied segment
    for i in range(start, end):
        if parent2[i] not in child:
            pos = i
            while True:
                mapped_city = parent1[pos]
                pos = parent2.index(mapped_city)
                if child[pos] == -1:
                    child[pos] = parent2[i]
                    break

    # Fill empty spaces with remaining cities from the second parent
    for i in range(size):
        if child[i] == -1:
            child[i] = parent2[i]
    return child

# Mutation function (reverse a subsequence)
def mutazione(percorso: list[str]) -> list[str]:
    i, j = sorted(random.sample(range(len(percorso)), 2))
    percorso[i:j+1] = reversed(percorso[i:j+1])
    return percorso

# Main function of the genetic algorithm with hybrid selection
def algoritmo_genetico(città: list[str], generations: int, population_size: int, 
                       initial_mutation_rate: float, min_mutation_rate: float, 
                       elitism_size: int) -> list[str]:
    
    popolazione = crea_popolazione(città, population_size)
    miglior_costo_globale = float('inf')
    
    for generazione in range(generations):
        popolazione.sort(key=lambda percorso: costo(percorso))
        
        # Keep the elite
        elite = popolazione[:elitism_size]
        nuova_generazione = elite[:]
        
        # Gradually update the mutation rate
        mutation_rate = max(min_mutation_rate, initial_mutation_rate * (1 - generazione / generations))
        
        # Generate a new population
        for _ in range(population_size - elitism_size):
            if random.random() < 0.5:
                parent1 = selezione_ranking(popolazione)
                parent2 = selezione_ranking(popolazione)
            else:
                parent1 = selezione_torneo(popolazione, SELECTION_SIZE)
                parent2 = selezione_torneo(popolazione, SELECTION_SIZE)
            
            child = pmx_ox_crossover(parent1, parent2)
            
            if random.random() < mutation_rate:
                child = mutazione(child)
            
            nuova_generazione.append(child)
        
        popolazione = nuova_generazione[:population_size]
        print(f"Generation {generazione}: Best cost = {costo(elite[0]):.2f} \t mutation rate={mutation_rate:.2f}", end='\r')
    
    return min(popolazione, key=lambda percorso: costo(percorso))

# Run the genetic algorithm
nomi_città = list(cities.keys())
miglior_percorso = algoritmo_genetico(nomi_città, NUM_GENERATIONS, POPULATION_SIZE, INITIAL_MUTATION_RATE, MIN_MUTATION_RATE, ELITISM_SIZE)

# Print the best path and cost
print("\nBest path (genetic algorithm):", miglior_percorso)
print("Path cost:", costo(miglior_percorso))
print("Valid solution:", is_valid_solution(miglior_percorso))


Generation 499: Best cost = 1345.54 	 mutation rate=0.10
Best path (genetic algorithm): ['Norsup', 'Luganville', 'Port Olry', 'Sola', 'Longana', 'Isangel', 'Vila', 'Lakatoro']
Path cost: 1345.544956473311
Valid solution: True
